# Over-Representation Analysis

Notebook for performing an over-representation analysis on the top and bottom strains, in terms of redness and normalized redness.

2020-02-26

## Initial boilerplate

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = ''
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))

print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


## 1. Load data

We will use:
* 4 lists (generated manually):
  * Top 500 genes in terms of redness, with size $\geq$ 180 and count > 1.
  * Top 500 genes in terms of normalized redness, with size $\geq$ 260.
  * Bottom 500 genes in terms of redness, with size $\geq$ 180 and count > 1.
  * Bottom 500 genes in terms of normalized redness, with size $\geq$ 180 and count > 1.
* The lookup table connecting positions in the 384-well plate to gene ids, as it has all GO-terms associated to each gene ID.

In [2]:
import pandas as pd

data = pd.read_csv(join(INTERMEDIATE,"top_and_bottom_500.csv"))
print(data)

    top_redness top_norm_redness bottom_redness bottom_norm_redness
0          CSE2             GAS5          IBA57                OPY1
1          MLC2             YOR1           OPY1                RIM8
2          YOR1          YJL175W        YCR090C               IBA57
3          NTA1             SKY1           HTD2                BUD8
4       YOL153C          YIL096C           DIA3               RPL9A
..          ...              ...            ...                 ...
495        PTK1             TRP1          SPO74                RTC2
496     YBR287W          YPR071W        YDL211C                IPK1
497        PLB2             BPT1           YCH1             YMR160W
498        TRP1             NEJ1           YSP2                STP2
499     YKR070W             REX3        YIL012W             YGL152C

[500 rows x 4 columns]


In [3]:
gene_ids = pd.read_csv(join(RAW_EXTERNAL,"geneIDs.txt"), sep="\t")
gene_ids = gene_ids[gene_ids["Gene"] != "Blank"]  # filter out blank data
gene_ids = gene_ids.drop(columns=["Plate #", "Row", "Column", "ORF", "96-position"])  # remove unused columns
print(gene_ids)

           Gene                                         Decription  \
1         VPS13  Protein of unknown function; heterooligomeric ...   
2          PAU8  Protein of unknown function, member of the ser...   
4          SEO1  Putative permease, member of the allantoate tr...   
5          SDH2  Iron-sulfur protein subunit of succinate dehyd...   
6       YAL066W  Dubious open reading frame unlikely to encode ...   
...         ...                                                ...   
4982       GLO3  ADP-ribosylation factor GTPase activating prot...   
4984      APQ13  Dubious open reading frame, unlikely to encode...   
4986       PDX3  Pyridoxine (pyridoxamine) phosphate oxidase, h...   
4988       NOT5  Subunit of the CCR4-NOT complex, which is a gl...   
4990  YOR008C-A  Putative protein of unknown function, includes...   

                                  GO Biological Process  \
1     transport, sporulation resulting in formation ...   
2     sporulation resulting in formation 

## 2. GO terms

We will first collect all GO terms in a single list, to be able to iterate through later.

In [4]:
GOterms = []
for (idx, row) in gene_ids.iterrows():
    row["GO Biological Process"] = row["GO Biological Process"].split(", ")
    GOterms.extend(row["GO Biological Process"])
GOterms = list(set(GOterms))
print(GOterms)
GOterms.remove('-')

['cell budding', 'chromosome organization', 'cytoskeleton organization', 'transport', 'mitochondrion organization', 'protein complex biogenesis', 'vacuole organization', 'cytokinesis', 'conjugation', 'protein modification process', 'translation', 'pseudohyphal growth', 'cellular component morphogenesis', 'cellular carbohydrate metabolic process', 'chromosome segregation', 'vesicle-mediated transport', 'fungal-type cell wall organization', '-', 'sporulation resulting in formation of a cellular spore', 'cellular respiration', 'cellular protein catabolic process', 'nucleus organization', 'meiosis', 'peroxisome organization', 'generation of precursor metabolites and energy', 'cellular homeostasis', 'ribosome biogenesis', 'signal transduction', 'heterocycle metabolic process', 'DNA metabolic process', 'protein folding', 'other', 'cellular aromatic compound metabolic process', 'cellular amino acid and derivative metabolic process', 'transposition', 'cell cycle', 'RNA metabolic process', 'cel

## 3. Over-Representation Analysis

We need a function for the over-representation analysis. For each GO term the Fisher matrix is constructed as:

|   | Selected in group | Not selected |
|:-:|:-----------------:|:------------:|
| With GO term | F[0][0] | F[0][1] |
| Without GO term | F[1][0] | F[1][1] |

And then the alternative hypothesis is that there is a higher percentage of genes in the selected group with the GO term Vs without the GO term.

In [5]:
from fisher import pvalue

# over-representation function:
def over_rep(group, gene_ids, GOterms):
    pvalues = pd.DataFrame()
    for GOterm in GOterms:
        F = [[0, 0], [0, 0]]
        for (idx, row) in gene_ids.iterrows():
            i = 0 if GOterm in row["GO Biological Process"] else 1
            j = 0 if row["Gene"] in group else 1
            F[i][j] += 1
        pval = pvalue(F[0][0], F[0][1], F[1][0], F[1][1]).right_tail
        new_row = pd.DataFrame(data={"genes.selected":F[0][0], "genes.total":F[0][0] + F[0][1], "p.value": pval}, index=[GOterm])
        pvalues = pvalues.append(new_row)
    return pvalues

Now we can call the function for all 4 groups:

In [6]:
top_redness = list(data["top_redness"])
p_values_top_redness = over_rep(top_redness, gene_ids, GOterms)
p_values_top_redness[p_values_top_redness["p.value"] < 0.05]

,genes.selected,genes.total,p.value


In [7]:
top_norm_redness = list(data["top_norm_redness"])
p_values_top_norm_redness = over_rep(top_norm_redness, gene_ids, GOterms)
p_values_top_norm_redness[p_values_top_norm_redness["p.value"] < 0.05]

,genes.selected,genes.total,p.value
biological_process,173,1405,0.004948


In [8]:
bottom_redness = list(data["bottom_redness"])
p_values_bottom_redness = over_rep(bottom_redness, gene_ids, GOterms)
p_values_bottom_redness[p_values_bottom_redness["p.value"] < 0.05]

,genes.selected,genes.total,p.value
protein modification process,56,410,0.026396


In [9]:
bottom_norm_redness = list(data["bottom_norm_redness"])
p_values_bottom_norm_redness = over_rep(bottom_norm_redness, gene_ids, GOterms)
p_values_bottom_norm_redness[p_values_bottom_norm_redness["p.value"] < 0.05]

,genes.selected,genes.total,p.value
vitamin metabolic process,9,46,0.049671


## 4. Save results

In [10]:
# Export datasets:
p_values_top_redness.to_csv(join(FINAL, "p_values_top_redness.csv"))
p_values_top_norm_redness.to_csv(join(FINAL, "p_values_top_norm_redness.csv"))
p_values_bottom_redness.to_csv(join(FINAL, "p_values_bottom_redness.csv"))
p_values_bottom_norm_redness.to_csv(join(FINAL, "p_values_bottom_norm_redness.csv"))